In [1]:
%config IPCompleter.greedy=True
from fastai.fastai.text import *
import pickle
from itertools import islice

In [2]:
Path.cwd()

PosixPath('/home/ubuntu/Project')

## Load Mapping

In [3]:
art_map = pd.read_csv("Transcript_Data/transcript_mapping.csv", index_col=0, parse_dates=[13])
art_map.head(5)

,gvkey,CUSIP,SEDOL,TIC,ISIN,transcript_id,factset_entity_Id,fsym_id,proper_name,fsym_primary_listing_id,ticker_region,company_id,event_date
0,002710:02,21036P108,2170473,STZ,US21036P1084,1048233,000CN7-E,QZ2Q0V-S,"Constellation Brands, Inc. Class A",RK3HS8-R,STZ-US,818,2007-01-04
1,008902:01,749685103,2756174,RPM,US7496851038,1047794,000VLT-E,DF312T-S,RPM International Inc.,LN3YSL-R,RPM-US,1299,2007-01-04
2,009472:01,808194104,2779342,SHLM,US8081941044,1048056,000VS7-E,TSDFC1-S,"A. Schulman, Inc.",CRJZ1Y-R,SHLM-US,2518,2007-01-04
3,010498:01,882491103,2885302,TXI,US8824911031,1047791,000XNX-E,PQH0NS-S,"Texas Industries, Inc.",KYD8YC-R,TXI-US,2768,2007-01-04
4,011914:01,452526106,2457875,BLUD,US4525261065,1048482,000M1P-E,W24LCD-S,"Immucor, Inc.",WXDDH4-R,BLUD-US,2665,2007-01-04


In [4]:
art_map = art_map[['transcript_id', 'gvkey', 'event_date']]
art_map.head(5)

,transcript_id,gvkey,event_date
0,1048233,002710:02,2007-01-04
1,1047794,008902:01,2007-01-04
2,1048056,009472:01,2007-01-04
3,1047791,010498:01,2007-01-04
4,1048482,011914:01,2007-01-04


In [5]:
art_map.shape

(64992, 3)

## Load Stock Return

In [5]:
ret = pd.read_csv("Transcript_Data/fwd_rets.csv", index_col=0, parse_dates=[2])
ret.head(5)

,gvkey,date,fwd_ret,fwd_ret_transformed,gics_sector,gics_industry_group,same_day_ret,fwd_ret_category,same_day_ret_category,one_day_ret_category,one_day_ret
0,140760:01,2007-01-04,0.011922,0.397977,15,1510,-0.010990,0,0,0.0,0.010084
1,002710:02,2007-01-04,0.000361,0.024577,30,3020,-0.114173,0,-1,-1.0,-0.074347
2,010498:01,2007-01-04,0.011307,0.359228,15,1510,0.037680,0,1,0.0,-0.000257
3,061732:01,2007-01-04,0.002841,0.094319,20,2010,0.057172,0,1,1.0,0.044503
4,146017:01,2007-01-04,0.100594,1.993171,20,2010,0.004846,1,0,-1.0,-0.031231


In [7]:
ret = ret[['gvkey', 'date', 'fwd_ret', 'fwd_ret_category']]
ret.head(5)

,gvkey,date,fwd_ret,fwd_ret_category
0,140760:01,2007-01-04,0.011922,0
1,002710:02,2007-01-04,0.000361,0
2,010498:01,2007-01-04,0.011307,0
3,061732:01,2007-01-04,0.002841,0
4,146017:01,2007-01-04,0.100594,1


In [6]:
ret['fwd_ret_category'].unique()

array([ 0,  1, -1])

In [7]:
print("before cleaning null: " + str(ret.shape))
ret = ret[pd.notnull(ret['fwd_ret']) & pd.notnull(ret['fwd_ret_category'])]
print("after cleaning null: " + str(ret.shape))

before cleaning null: (79666, 11)
after cleaning null: (79600, 11)


## Make The Split

In [14]:
cvp = pd.read_csv("Transcript_Data/cv_periods.csv", index_col=0, parse_dates=[1,2,3,4])
# cvp

,cv-start,cv-through,train-end,train-start
0,2011-01-04,2011-07-01,2010-12-20,2007-02-07
1,2011-07-07,2012-01-01,2011-06-22,2007-08-10
2,2012-01-04,2012-07-01,2011-12-20,2008-02-07
3,2012-07-02,2013-01-01,2012-06-15,2008-08-05
4,2013-01-03,2013-07-01,2012-12-19,2009-02-06
5,2013-07-02,2014-01-01,2013-06-17,2009-08-05
6,2014-01-03,2014-07-01,2013-12-19,2010-02-08
7,2014-07-02,2015-01-01,2014-06-17,2010-08-05
8,2015-01-06,2015-07-01,2014-12-22,2011-02-09
9,2015-07-02,2015-12-31,2015-06-17,2011-08-05


In [18]:
for fold_index in range(len(cvp)):
    row = cvp.iloc[fold_index]
    train_ret = ret[
        (ret['date'] >= pd.Timestamp(row['train-start']))
        &
        (ret['date'] <= pd.Timestamp(row['train-end']))
    ]

    test_ret = ret[
        (ret['date'] >= pd.Timestamp(row['cv-start']))
        &
        (ret['date'] <= pd.Timestamp(row['cv-through']))
    ]
    
    print("train all: " + str(train_ret.shape))
    print("test all: " + str(test_ret.shape))
    
    train_ret.to_csv('results_rakesh/rets_folds/train_f'+str(fold_index+1)+'.csv')
    test_ret.to_csv('results_rakesh/rets_folds/test_f'+str(fold_index+1)+'.csv')
    
    # remove class 0
    train_ret = train_ret[train_ret["fwd_ret_category"].isin([-1, 1])]
    test_ret = test_ret[test_ret["fwd_ret_category"].isin([-1, 1])]
    train_ret["fwd_ret_category"][train_ret["fwd_ret_category"] == -1] = 0
    test_ret["fwd_ret_category"][test_ret["fwd_ret_category"] == -1] = 0
    #
    print("train 01: " + str(train_ret.shape))
    print("test 01: " + str(test_ret.shape))
    train_ret.to_csv('results_rakesh/rets_folds_binary/train_f'+str(fold_index+1)+'.csv')
    test_ret.to_csv('results_rakesh/rets_folds_binary/test_f'+str(fold_index+1)+'.csv')
        

train all: (33419, 11)
test all: (4254, 11)
train 01: (19733, 11)
test 01: (2427, 11)
train all: (31918, 11)
test all: (4261, 11)
train 01: (18706, 11)
test 01: (2491, 11)
train all: (32461, 11)
test all: (4199, 11)
train 01: (19042, 11)
test 01: (2405, 11)
train all: (31624, 11)
test all: (4025, 11)
train 01: (18502, 11)
test 01: (2332, 11)
train all: (31694, 11)
test all: (4236, 11)
train 01: (18458, 11)
test 01: (2480, 11)
train all: (31805, 11)
test all: (4484, 11)
train 01: (18521, 11)
test 01: (2658, 11)
train all: (32845, 11)
test all: (4735, 11)
train 01: (19180, 11)
test 01: (2767, 11)
train all: (32969, 11)
test all: (4940, 11)
train 01: (19204, 11)
test 01: (2842, 11)
train all: (34225, 11)
test all: (4989, 11)
train 01: (19926, 11)
test 01: (2878, 11)
train all: (34264, 11)
test all: (5104, 11)
train 01: (19961, 11)
test 01: (2978, 11)


## Load Article

In [19]:
art_path = Path("Transcript_Data/transcripts.pkl")
art_dict = pickle.load(open(art_path, "rb"))

In [20]:
art_map = pd.read_csv("Transcript_Data/transcript_mapping.csv", index_col=0, parse_dates=[13])
art_map.head(5)

,gvkey,CUSIP,SEDOL,TIC,ISIN,transcript_id,factset_entity_Id,fsym_id,proper_name,fsym_primary_listing_id,ticker_region,company_id,event_date
0,002710:02,21036P108,2170473,STZ,US21036P1084,1048233,000CN7-E,QZ2Q0V-S,"Constellation Brands, Inc. Class A",RK3HS8-R,STZ-US,818,2007-01-04
1,008902:01,749685103,2756174,RPM,US7496851038,1047794,000VLT-E,DF312T-S,RPM International Inc.,LN3YSL-R,RPM-US,1299,2007-01-04
2,009472:01,808194104,2779342,SHLM,US8081941044,1048056,000VS7-E,TSDFC1-S,"A. Schulman, Inc.",CRJZ1Y-R,SHLM-US,2518,2007-01-04
3,010498:01,882491103,2885302,TXI,US8824911031,1047791,000XNX-E,PQH0NS-S,"Texas Industries, Inc.",KYD8YC-R,TXI-US,2768,2007-01-04
4,011914:01,452526106,2457875,BLUD,US4525261065,1048482,000M1P-E,W24LCD-S,"Immucor, Inc.",WXDDH4-R,BLUD-US,2665,2007-01-04


In [21]:
len(art_dict)

84399

In [22]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))
take(5, art_dict.items())

[(1572864,
  {'text': "\n    \n    \n    I would now like to introduce today's host, Mr. Steve Eschbach, Vice President of Investor Relations at Integrys Energy Group. Sir, you may now begin.\n    \n    \n    \n    \n    Before we begin, I will advise everyone that this call is being recorded and will be available for audio replay through November 5, 2013.\n    I will now turn this call over to Charlie Schrock.\n    \n    \n    \n    \n    Thanks, Steve. Good morning, everyone, and thanks for joining us on the call today. I'll provide an overview of our second quarter and our year-to-date financial results, some recent operational highlights and our expectations for the balance of 2013.\n    Jim Schott will discuss our financial results in more detail, and he will provide more information regarding our financial outlook. And as usual, we will conclude with a question-and-answer session.\n    Turning to slide 5, diluted EPS adjusted was up in 2013 compared with 2012 for both the quarter

In [23]:
def create_one_doc_df(ret_df):
    labels = []
    returns = []
    art_ids = []
    gvkeys = []
    dates = []
    docs = []
    for i, row in ret_df.iterrows():
        label = row['fwd_ret_category']
        ret_v = row['fwd_ret']
        gvkey = row['gvkey']
        date = row['date']
        matched_art = art_map[(art_map['gvkey'] == gvkey) & (art_map['event_date'] == date)]
#         print(matched_art)
        for art_id in matched_art['transcript_id']:
            if art_id in art_dict:
                labels.append(label)
                returns.append(ret_v)
                art_ids.append(art_id)
                gvkeys.append(gvkey)
                dates.append(date)
                docs.append(" -text- " + art_dict[art_id]["text"].replace("\n", " "))
        if i % 1000 == 0:
            print(str(i) + " " + str(len(ret_df)))
    df = pd.DataFrame({"label":labels, "ret":returns, "art_id":art_ids, "gvkey":gvkeys, "date":dates, "doc":docs})
    return df

## Convert 01 Files

In [24]:
# this is to attach a document to each row of the rets df

for fold_index in range(1,10):
    print("fold: " + str(fold_index + 1))
    train_ret = pd.read_csv('results_rakesh/rets_folds/train_f'+str(fold_index+1)+'.csv')
    test_ret = pd.read_csv('results_rakesh/rets_folds/test_f'+str(fold_index+1)+'.csv')

    train_df = create_one_doc_df(train_ret)
    test_df = create_one_doc_df(test_ret)
    print(train_df.shape)
    print(test_df.shape)
    
    train_df.to_csv('results_rakesh/one_doc_df/train_f'+str(fold_index+1)+'.csv', index=False)
    test_df.to_csv('results_rakesh/one_doc_df/test_f'+str(fold_index+1)+'.csv', index=False)


fold: 2
0 31918
1000 31918
2000 31918
3000 31918
4000 31918
5000 31918
6000 31918
7000 31918
8000 31918
9000 31918
10000 31918
11000 31918
12000 31918
13000 31918
14000 31918
15000 31918
16000 31918
17000 31918
18000 31918
19000 31918
20000 31918
21000 31918
22000 31918
23000 31918
24000 31918
25000 31918
26000 31918
27000 31918
28000 31918
29000 31918
30000 31918
31000 31918
0 4261
1000 4261
2000 4261
3000 4261
4000 4261
(25081, 6)
(3507, 6)
fold: 3
0 32461
1000 32461
2000 32461
3000 32461
4000 32461
5000 32461
6000 32461
7000 32461
8000 32461
9000 32461
10000 32461
11000 32461
12000 32461
13000 32461
14000 32461
15000 32461
16000 32461
17000 32461
18000 32461
19000 32461
20000 32461
21000 32461
22000 32461
23000 32461
24000 32461
25000 32461
26000 32461
27000 32461
28000 32461
29000 32461
30000 32461
31000 32461
32000 32461
0 4199
1000 4199
2000 4199
3000 4199
4000 4199
(25811, 6)
(3504, 6)
fold: 4
0 31624
1000 31624
2000 31624
3000 31624
4000 31624
5000 31624
6000 31624
7000 31624
8

In [25]:
train_df.shape

(29151, 6)

In [26]:
test_df.shape

(4427, 6)

In [27]:
test_df.head(1)

,label,ret,art_id,gvkey,date,doc
0,0,0.008755,1756432,024352:01,2015-07-02,"-text- At this time, I'd like ..."


## Convert ALL TEST Files

In [30]:
# for fold_index in range(1,10):
#     print("fold: " + str(fold_index + 1))
#     train_ret = pd.read_csv('Transcript_Data/10day/ret_all/ret_all_train_f'+str(fold_index+1)+'.csv')
#     test_ret = pd.read_csv('Transcript_Data/10day/ret_all/ret_all_test_f'+str(fold_index+1)+'.csv')

#     train_df = create_one_doc_df(train_ret)
#     test_df = create_one_doc_df(test_ret)
#     print(train_df.shape)
#     print(test_df.shape)
    
#     train_df.to_csv('Transcript_Data/10day/cls_10d_all/train_f'+str(fold_index+1)+'.csv', index=False)
#     test_df.to_csv('Transcript_Data/10day/cls_10d_all/test_f'+str(fold_index+1)+'.csv', index=False)

# TOKENIZATION

In [33]:
chunksize=24000

In [34]:
# modify numbers in the text

def fixup(x):
    
    x = re.sub(r'(,[\d][\d]{2})', ' 000 ', x) # $605,100,100 -> $605 000  000
    
    x = re.sub('([\D][0]\.[1-9])[\d]+', r'\1', x)  # 0.59 -> 0.5
    x = re.sub(r'([\D][1-9])\.[\d]+', r'\g<1>', x)  # 3.6 -> 3
    x = re.sub(r'([\D^\.][1][0-9])\.[\d]+', r'\g<1>', x)  # 19.6 -> 19
    x = re.sub(r'([\D^\.][2-9])[0-9](\.[\d]+)?([\D])', r'\g<1>0\g<3>', x)  # 26.25 -> 26
    x = re.sub(r'([\D^\.][\d])[\d]{2}(\.[\d]+)?([\D])', r'\g<1>00\g<3>', x)  # $158.4 million -> $100 million
    
    x = re.sub(r'\$', r' $ ', x)
    x = re.sub(r'\%', r' % ', x)
    
    x = x.replace('pppp', 'xppx')
    
    return x

In [35]:
# tokenization of text - one fold df at a time

def get_texts(df):
    labels = df.iloc[:,0].values.astype(np.int64)
    rets = df.iloc[:,1].values.astype(np.float64)
    aids = df.iloc[:,2].values.astype(str)
    gvkeys = df.iloc[:,3].values.astype(str)
    dates = df.iloc[:,4].values.astype(str)
    texts = df.iloc[:,5].astype(str)
    
    tok = []    
    texts = list(texts.apply(fixup).values)
    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok, list(labels), list(rets), list(aids), list(gvkeys), list(dates)

In [46]:
# convert all tokenized data to lists

def get_all(df):
    tok = []
    labels = []
    returns = []
    aids = []
    gvkeys = []
    dates = []
    for i, r in enumerate(df):
        print(i)
        tok_, labels_, rets_, aids_, gvkeys_, dates_ = get_texts(r)
        tok += tok_
        labels += labels_
        returns += rets_
        aids += aids_
        gvkeys += gvkeys_
        dates += dates_
    return tok, labels, returns, aids, gvkeys, dates

In [36]:
# load word index and vocab from when the language model was trained

itos = pickle.load(Path('Transcript_Data/itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

60002

In [41]:
take(20, stoi.items())

[('_unk_', 0),
 ('_pad_', 1),
 ('the', 2),
 ('.', 3),
 (',', 4),
 ('to', 5),
 ('and', 6),
 ('of', 7),
 ('in', 8),
 ('our', 9),
 ('we', 10),
 ('crlf', 11),
 ('a', 12),
 ('for', 13),
 ('quarter', 14),
 ('$', 15),
 ('-', 16),
 ('%', 17),
 ('that', 18),
 ('million', 19)]

In [42]:
itos[:20]

['_unk_',
 '_pad_',
 'the',
 '.',
 ',',
 'to',
 'and',
 'of',
 'in',
 'our',
 'we',
 'crlf',
 'a',
 'for',
 'quarter',
 '$',
 '-',
 '%',
 'that',
 'million']

## CONVERTING 01 DATA

In [47]:
# this reads the one_doc in and converts into another df of labels? redundant to save?
# it also maps the tokenized text to indices - how is unk assigned?

for fold_index in range(1, 10):
    print("fold: " + str(fold_index+1))
    df_train = pd.read_csv('results_rakesh/one_doc_df/train_f' + str(fold_index+1) + '.csv',
                           chunksize=chunksize, skiprows=1, header=None)
    df_test = pd.read_csv('results_rakesh/one_doc_df/test_f' + str(fold_index+1) + '.csv',
                          chunksize=chunksize, skiprows=1, header=None)
    
    tok_train, lbl_train, ret_train, aids_train, gvkeys_train, dates_train = get_all(df_train)
    tok_test, lbl_test, ret_test, aids_test, gvkeys_test, dates_test = get_all(df_test)
    
    # form info file
    df = pd.DataFrame({"label":lbl_train, "1day_ret":ret_train, "article_id":aids_train,
                       "gvkey":gvkeys_train, "date":dates_train})
    df.to_csv('results_rakesh/one_doc_df/doc_info_train_f' + str(fold_index+1) + '.csv')
    df = pd.DataFrame({"label":lbl_test, "1day_ret":ret_test, "article_id":aids_test,
                       "gvkey":gvkeys_test, "date":dates_test})
    df.to_csv('results_rakesh/one_doc_df/doc_info_test_f' + str(fold_index+1) + '.csv')
    
    # form ids file
    cls_train = np.array([[stoi[o] for o in p] for p in tok_train])
    cls_test = np.array([[stoi[o] for o in p] for p in tok_test])
    
    np.save('results_rakesh/tok_inds_npy/cls_ids_train_f' + str(fold_index+1) + '.npy', cls_train)
    np.save('results_rakesh/tok_inds_npy/cls_ids_test_f' + str(fold_index+1) + '.npy', cls_test)

fold: 2
0
1
0
fold: 3
0
1
0
fold: 4
0


Process Process-26:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/concurrent/futures/process.py", line 181, in _process_worker
    result=r))
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/multiprocessing/queues.py", line 341, in put
    obj = _ForkingPickler.dumps(obj)
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
MemoryError


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

## CONVERTING ALL TEST DATA

In [ ]:
# df_test = pd.read_csv('Transcript_Data/10day/cls_10d_all/test_f1.csv')
# df_test.head(3)

In [ ]:
# for fold_index in range(1,10):
#     print("fold: " + str(fold_index+1))
#     df_train = pd.read_csv('Transcript_Data/10day/cls_10d_all/train_f' + str(fold_index+1) + '.csv',
#                            chunksize=chunksize, skiprows=1, header=None)
#     df_test = pd.read_csv('Transcript_Data/10day/cls_10d_all/test_f' + str(fold_index+1) + '.csv',
#                           chunksize=chunksize, skiprows=1, header=None)
    
#     tok_train, lbl_train, ret_train, aids_train, gvkeys_train, dates_train = get_all(df_train)
#     tok_test, lbl_test, ret_test, aids_test, gvkeys_test, dates_test = get_all(df_test)
#     # form info file
#     df = pd.DataFrame({"label":lbl_train, "ret":ret_train, "article_id":aids_train,
#                        "gvkey":gvkeys_train, "date":dates_train})
#     df.to_csv('Transcript_Data/10day/tok_10d_all/doc_info_train_f' + str(fold_index+1) + '.csv')
#     df = pd.DataFrame({"label":lbl_test, "ret":ret_test, "article_id":aids_test,
#                        "gvkey":gvkeys_test, "date":dates_test})
#     df.to_csv('Transcript_Data/10day/tok_10d_all/doc_info_test_f' + str(fold_index+1) + '.csv')
#     # form ids file
#     cls_train = np.array([[stoi[o] for o in p] for p in tok_train])
#     np.save('Transcript_Data/10day/tok_10d_all/cls_ids_train_f' + str(fold_index+1) + '.npy', cls_train)
            
#     cls_test = np.array([[stoi[o] for o in p] for p in tok_test])
#     np.save('Transcript_Data/10day/tok_10d_all/cls_ids_test_f' + str(fold_index+1) + '.npy', cls_test)

In [48]:
cls_train.shape

(25811,)